# Labelizer

Script para etiquetar los datos de auctions, y guardarlos en un csv aparte.

In [1]:
import pandas as pd

## St(d)

**Importante:** Sacar nrows para procesar el archivo completo

In [2]:
auctions = pd.read_csv('data/auctions.csv', dtype={'date': str, 'ref_type_id': 'category', 'source_id': 'category'}, parse_dates=['date'], nrows=10000)

Primero ordeno por fecha

In [3]:
auctions = auctions.sort_values(by='date')

Agrupo por dispositivo y para cada uno de ellos me fijo la diferencia de cada valor de la columna 'date' con la columna siguiente. Esa diferencia la agrego como una nueva columna.

In [4]:
auctionsGroupBy = auctions.groupby(['ref_type_id', 'device_id'])
labeledAuctions = auctionsGroupBy.apply(lambda x: pd.concat([x, -x['date'].diff(periods=-1).rename('s_t')], axis=1))

Podemos ver que funciono si tomamos un dispositivo que aparezca mas de una vez, y vemos sus valores

In [5]:
device = labeledAuctions['device_id'].value_counts().index[0]

In [6]:
labeledAuctions[labeledAuctions['device_id'] == device].head()

,date,device_id,ref_type_id,source_id,s_t
6017,2019-04-23 21:21:35.919273,1152087653449695535,1,3,00:00:22.263941
6060,2019-04-23 21:21:58.183214,1152087653449695535,1,3,00:00:03.144650
6067,2019-04-23 21:22:01.327864,1152087653449695535,1,3,00:00:07.378871
6085,2019-04-23 21:22:08.706735,1152087653449695535,1,5,00:00:47.275522
6187,2019-04-23 21:22:55.982257,1152087653449695535,1,3,00:00:01.720101


Se ve que en la columna s_t esta el tiempo que falta para el date de la siguiente fila

Por ultimo vuelvo al orden original

In [7]:
labeledAuctions = labeledAuctions.sort_index()

## Sc(d)

In [8]:
installs = pd.read_csv('data/installs.csv', usecols=['created', 'ref_type', 'ref_hash'], dtype={'created': str, 'ref_type': 'category'}, parse_dates=['created'])

Para cada fila de auctions, busco la minima fecha de instalacion de ese dispositivo, siempre que sea mayor que la fecha del auction. No se si existe una forma mas 'pandas' de hacer esto, asi es como se me ocurrio a mi.

In [9]:
next_install_date = pd.Series()
for index, row in auctions.iterrows():
    next_install_date.at[index] = installs[(installs['ref_type'] == row['ref_type_id']) & (installs['ref_hash'] == row['device_id']) & (installs['created'] > row['date'])]['created'].min()

Ahora calculo Sc(d) restando a la fecha de conversion la fecha del auction.

In [10]:
labeledAuctions['s_c'] = next_install_date - auctions['date']

In [11]:
labeledAuctions.head()

,date,device_id,ref_type_id,source_id,s_t,s_c
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0,NaT,NaT
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0,00:10:08.526877,NaT
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0,00:52:38.441254,NaT
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0,NaT,NaT
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0,NaT,NaT


Por ultimo, genero un nuevo csv con los datos etiquetados

In [12]:
labeledAuctions.to_csv('data/labeled_auctions.csv', index=False)